## Custom Fuzzy + Semantic Match
Tryed using Word2Vec Cosine similarity in combination with fuzzywuzzy partial fuzzy match to get closest matches. Unsatisfactory results.

In [ ]:
!pip install fuzzywuzzy
!pip install gensim

In [ ]:
import pandas as pd
import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim import models

In [ ]:
# Downloading word2vec embeddings
word_vectors = api.load("word2vec-google-news-300")

Creating combined dataset for Categories, Brands and Labels

In [ ]:
df_categories = pd.read_csv('categories.csv')
df_brands = pd.read_csv('brand_category.csv')
df_offers = pd.read_csv('offer_retailer.csv')

In [ ]:
def preprocess_column(column):
    column = column.astype(str).str.lower()
    column = column.str.replace(r'-', ' ', regex=True)
    column = column.str.replace(r'[^a-z0-9\s]', '', regex=True)
    column = column.str.replace(r'\s+', ' ', regex=True)
    column = column.str.strip()
    return column

In [ ]:
df_categories['PRODUCT_CATEGORY'] = preprocess_column(df_categories['PRODUCT_CATEGORY'])
df_categories['IS_CHILD_CATEGORY_TO'] = preprocess_column(df_categories['IS_CHILD_CATEGORY_TO'])

df_brands['BRAND'] = preprocess_column(df_brands['BRAND'])
df_brands['BRAND_BELONGS_TO_CATEGORY'] = preprocess_column(df_brands['BRAND_BELONGS_TO_CATEGORY'])

df_offers['OFFER_PREPROCESSED'] = preprocess_column(df_offers['OFFER'])
df_offers['RETAILER'] = preprocess_column(df_offers['RETAILER'])
df_offers['BRAND'] = preprocess_column(df_offers['BRAND'])

In [ ]:
# df_categories

In [ ]:
# df_brands

In [ ]:
# df_offers

In [ ]:
category_list = list(df_categories['PRODUCT_CATEGORY'])
category_list += list(df_brands['BRAND_BELONGS_TO_CATEGORY'])
category_list = list(set(category_list))

In [ ]:
brand_list = list(df_brands['BRAND']) + list(df_offers['BRAND'])
brand_list = list(set(brand_list))

In [ ]:
retailer_list = list(df_offers['RETAILER'])
retailer_list = list(set(retailer_list))

In [ ]:
len(category_list)

118

In [ ]:
len(brand_list)

8417

In [ ]:
len(retailer_list)

62

In [ ]:
# Create a list of tuples with elements and their source list names
combined_data = [(item, 'category_list') for item in category_list] + \
               [(item, 'brand_list') for item in brand_list] + \
               [(item, 'retailer_list') for item in retailer_list]

# Create a DataFrame from the combined data
df = pd.DataFrame(combined_data, columns=['Element', 'Source'])

In [ ]:
df

,Element,Source
0,frozen,category_list
1,nail care,category_list
2,nuts seeds,category_list
3,pretzels,category_list
4,oral care,category_list
...,...,...
8592,pet supplies plus,retailer_list
8593,tom thumb,retailer_list
8594,burger king,retailer_list
8595,nekter juice bar,retailer_list


In [ ]:
def normalize_scores(scores):
    min_score = min(scores, key=lambda x: x[2])[2]
    max_score = max(scores, key=lambda x: x[2])[2]

    return [[item[0], item[1], (item[2] - min_score) / (max_score - min_score)] for item in scores]

In [ ]:
from fuzzywuzzy import fuzz

def keyword_search(input_str):
    fuzz_pairs = []
    semantic_pairs = []

    for index, row in df.iterrows():
        # string_tokens = row['Element'].lower().split()

        fuzz_score = fuzz.partial_ratio(input_str, row['Element'])
        semantic_score = word_vectors.n_similarity(input_str, row['Element'])
        # semantic_score = semantic_match(input_str, row['Element'])

        if not(len(row['Element']) < 5 and abs(len(row['Element']) - len(input_str)) > 4):
            fuzz_pairs.append([row['Element'], row['Source'], fuzz_score])
        semantic_pairs.append([row['Element'], row['Source'], int(semantic_score*100)])

    fuzz_pairs = sorted(fuzz_pairs, key=lambda x: x[2], reverse=True)
    semantic_pairs = sorted(semantic_pairs, key=lambda x: x[2], reverse=True)

    fuzz_pairs = normalize_scores(fuzz_pairs)
    semantic_pairs = normalize_scores(semantic_pairs)

    return fuzz_pairs, semantic_pairs



/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def fetch_results(search_results, input_str):
    offers = []

    for item in search_results:
        # print(item)

        if item[1] == 'category_list':
            brands = []
            brands = list(df_brands[df_brands['BRAND_BELONGS_TO_CATEGORY'] == item[0]]['BRAND'])

            for i in range(len(df_offers['BRAND'])):
                if df_offers['BRAND'][i] in brands:
                    offers.append(df_offers['OFFER'][i])

        elif item[1] == 'brand_list':
            for i in range(len(df_offers['BRAND'])):
                if df_offers['BRAND'][i] == item[0]:
                    offers.append(df_offers['OFFER'][i])

        elif item[1] == 'retailer_list':
            offers.extend(list(df_offers[df_offers['RETAILER'] == item[0]]['OFFER']))

    for i in range(len(df_offers)):
        if word_vectors.n_similarity(item[0], df_offers['OFFER_PREPROCESSED'][i]) > 0.97:
            offers.append(df_offers['OFFER'][i])

    # for i in range(len(offers)):

    return list(set(offers))


In [ ]:
df_offers

In [ ]:
# Example usage:
INPUT_STRING = "diapers"  # Replace with your input string
r1, r2 = keyword_search(INPUT_STRING)
top_keywords = r1 + r2
top_keywords = sorted(top_keywords, key=lambda x: x[2], reverse=True)


In [ ]:
r1[:20]

[['diapering', 'category_list', 1.0],
 ['persil', 'brand_list', 0.9302325581395349],
 ['powers', 'brand_list', 0.9302325581395349],
 ['petrus', 'brand_list', 0.9302325581395349],
 ['apex', 'brand_list', 0.872093023255814],
 ['dial', 'brand_list', 0.872093023255814],
 ['jb raders', 'brand_list', 0.8488372093023255],
 ['kinders', 'brand_list', 0.8488372093023255],
 ['walders', 'brand_list', 0.8488372093023255],
 ['founders', 'brand_list', 0.8488372093023255],
 ['jks farmhouse ciders', 'brand_list', 0.8488372093023255],
 ['stem ciders', 'brand_list', 0.8488372093023255],
 ['marie callenders', 'brand_list', 0.8488372093023255],
 ['birch benders', 'brand_list', 0.8488372093023255],
 ['chex mix muddy buddies', 'brand_list', 0.8488372093023255],
 ['hungbloodies', 'brand_list', 0.8488372093023255],
 ['austin eastciders', 'brand_list', 0.8488372093023255],
 ['bar tenders', 'brand_list', 0.8488372093023255],
 ['graft green planet ciders', 'brand_list', 0.8488372093023255],
 ['blondies', 'brand_l

In [ ]:
r2[:20]

[['paradise ciders', 'brand_list', 1.0],
 ['red stripe', 'brand_list', 0.9878048780487805],
 ['prairie pineapple upside down cake sour ale',
  'brand_list',
  0.975609756097561],
 ['cote de paradis', 'brand_list', 0.975609756097561],
 ['persil', 'brand_list', 0.975609756097561],
 ['wild state pineapple cider', 'brand_list', 0.975609756097561],
 ['superbird', 'brand_list', 0.975609756097561],
 ['sharpie', 'brand_list', 0.975609756097561],
 ['sierra nevada tropical torpedo ipa', 'brand_list', 0.975609756097561],
 ['somersby hard apple cider', 'brand_list', 0.975609756097561],
 ['piper heidsieck', 'brand_list', 0.975609756097561],
 ['mikes hard pineapple strawberry', 'brand_list', 0.975609756097561],
 ['striped pig distillery', 'brand_list', 0.975609756097561],
 ['freeland spirits', 'brand_list', 0.975609756097561],
 ['paradise island', 'brand_list', 0.975609756097561],
 ['pressure drop', 'brand_list', 0.975609756097561],
 ['pro pride', 'brand_list', 0.975609756097561],
 ['dog supplies', 

In [ ]:
ans = fetch_results(top_keywords[:50], INPUT_STRING)

In [ ]:
ans

["Rao's®, Pasta OR Sauce, spend $10",
 'Kradle, select varieties, at Pet Supplies Plus',
 "Michael Angelo's® OR Rao's® Frozen Entrees, spend $10",
 'Barilla® pasta, select varieties, buy 4',
 'CESAR® Wet Dog Food Singles',
 "Ben & Jerry's®, select varieties",
 'Kradle, select products, online at Chewy.com',
 "Select Beverages AND Prepared Food Item at Casey's",
 'Barilla® Pasta, select varieties, buy 2',
 'Butterball, select varieties, spend $10 at Fred Meyer',
 "Rao's® Frozen Pizza",
 'Sargento Sliced Cheese, spend $12',
 'Spend $5 on brisket mac & cheese sandwich OR other single-serve prepared food item',
 "Breyers Reese's Ice Cream, select varieties, at GIANT OR MARTIN’S",
 'Perfect Keto Super Reds, online at Amazon',
 "Spend $5 on single-serve prepared food items at Casey's",
 "Select beverages AND prepared food items at Casey's",
 'Purex® laundry detergent, select varieties, at Walmart',
 "Ben & Jerry's®, select varieties, spend $12",
 'Kradle, select varieties, spend $20 at Walma

In [ ]:
len(set(ans))

63